In [1]:
import os, sys
from pathlib import Path

src_path = Path(os.getcwd()).resolve().parents[1]  
sys.path.append(str(src_path))

In [2]:
import re
import numpy as np
import base64
import requests
from tqdm import tqdm
from PIL import Image
import fitz
import io
import pandas as pd
import random
import ast
from openai import OpenAI
import json
import fitz
import xml.etree.ElementTree as ET
import xml.dom.minidom as minidom
import base64
gpt = OpenAI()

In [4]:
from langchain_core.documents import Document
from utils.util import get_project_root
import re
file_path = get_project_root() / "src" / "data" / "housing_pre_promise" / "housing_faq_chunks"


def load_txt_as_documents(path: str):
    docs = []
    with open(path, "r", encoding="utf-8-sig") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue  # 빈 줄은 건너뜀
            docs.append(Document(page_content=line))
    return docs

docs = load_txt_as_documents(file_path)

# 1. 주택 청약 FAQ 문서 - 파싱 

In [5]:
from langchain_community.vectorstores import PGVector
from langchain_openai import OpenAIEmbeddings
import os 
from tools.rag.db_collection_name import HOUSING_FAQ_KEY
emb = OpenAIEmbeddings(model = "text-embedding-3-large")
connection_url = os.getenv("POSTGRES_URL")
store = PGVector.from_documents(
        documents= docs,
        embedding= emb, 
        connection_string = connection_url,
        collection_name= HOUSING_FAQ_KEY,
        use_jsonb=True,
        pre_delete_collection=False
)